Import required packages

In [7]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import linear_kernel
from copy import deepcopy
from scipy.spatial.distance import pdist, squareform
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
from fuzzywuzzy import process
from sklearn.model_selection import train_test_split

Read the file

In [2]:
file = pd.read_excel('shortcourses2566.xlsx')

Count the number of courses

In [3]:
course_counts = pd.DataFrame(file)['หลักสูตรอบรมระยะสั้น'].value_counts()
courses = pd.Series(course_counts.index)
courses = courses.sort_values().set_axis(range(0,len(courses)))

Create Series of users

In [4]:
s_name = file.loc[:, 'ชื่อ-นามสกุล (อังกฤษ)']
Users = pd.Series(s_name, name='User')
Users

0             PORPHAING JANTIP
1         THANAKORN DARASRISAK
2             KARNJANA EAMTANG
3             KARNJANA EAMTANG
4       THITIMUN VORATHONGCHAI
                 ...          
6122         KITTINON SANTASUP
6123      WITTAWAT SERMSRIPONG
6124               NON NAKKARA
6125         PHATTHARAPHON LIN
6126      THANAPAT SEEMAKAJOHN
Name: User, Length: 6127, dtype: object

Create Series of emails

In [5]:
s_email = file.loc[:, 'อีเมล'].fillna("")
Emails = pd.Series(s_email ,name='Email')
Emails

0            tanghaoren17@gamil.com
1               tuakung21@gmail.com
2            karnjana.aon@gmail.com
3            karnjana.aon@gmail.com
4          thitimun.rama1@gmail.com
                   ...             
6122           kittinon_s@cmu.ac.th
6123        wittawat_serm@cmu.ac.th
6124          non_nakkara@cmu.ac.th
6125    phattharaphon_lin@cmu.ac.th
6126         thanapat_see@cmu.ac.th
Name: Email, Length: 6127, dtype: object

Provide a score to each user based on their email domain

In [6]:
email_score = []
for data in Emails:
    if data != '':
        if data.split('@')[1] == 'cmu.ac.th':
            email_score.append(2)
        else:
            email_score.append(1)
    else:
        email_score.append(0)
email_score = pd.Series(email_score, name='Score Email')

email_score

0       1
1       1
2       1
3       1
4       1
       ..
6122    2
6123    2
6124    2
6125    2
6126    2
Name: Score Email, Length: 6127, dtype: int64

Email Score Statistic

In [7]:
zero_score_count = email_score.where(email_score == 0).count()
one_score_count = email_score.where(email_score == 1).count()
two_score_count = email_score.where(email_score == 2).count()

print("Number of students who fill cmu email:", two_score_count)
print("Number of students who fill other email:", one_score_count)
print("Number of students who do not fill email:", zero_score_count)

Number of students who fill cmu email: 1268
Number of students who fill other email: 4853
Number of students who do not fill email: 6


Create function to calculate age-education score

In [8]:
def getAgeEducationScore(age, limit_age):
    if age <= limit_age:
        score = 1
    elif limit_age == 0:
        score = 0
    else:
        score = 3
    return score

Create set of the educational range

In [9]:
set_nan = {'อื่นๆ (-)', np.nan}
set_primaryschool = {'ประถมศึกษา', 'อื่นๆ (ป.4)', 'อื่นๆ (ป.7)', 'อื่นๆ (ป7)'}
set_middleschool = {'มัธยมศึกษาตอนต้น', 'Secondary school', 'อื่นๆ (มศ.3)'}
set_highschool = {'มัธยมศึกษาตอนปลาย', 'High school', 'Vocational', 'การศึกษานอกระบบ', 
                  'ประกาศนียบัตรวิชาชีพ (ปวช.)', 'อื่นๆ (ม.ปลาย จบหลักสูตรEMR เป็นจนท.ปฏิบัติการ)',
                  'อื่นๆ (กำลังศึกษาชั้นมัธยมศึกษาตอนปลาย)', 'อื่นๆ (กำลังศึกษาชั้นมัธยมศึกษาปีที่6)', 
                  'อื่นๆ (มศ.5)'}
set_bachelor = {'ปริญญาตรี', 'Bachelor degree', 'Diploma', 'High Vocational', 
                'ประกาศนียบัตรวิชาชีพชั้นสูง (ปวส.)', 'อื่นๆ (กำลังศึกษาในระดับปริญญาตรี)', 
                'อื่นๆ (กำลังศึกษาปริญญาตรี สาขารังสีเทคนิค)', 'อื่นๆ (ปริญญาแพทยศาสตร์บัณฑิต)', 
                'อื่นๆ (นักศึกษาแพทย์ปี 5)', 'อื่นๆ (นักศึกษาแพทย์ มช ปี4 ศูนย์เชียงราย)', 
                'อื่นๆ (แพทยศาสตร์บัณฑิต)', 'อื่นๆ (แพทย์)', 'อื่นๆ (ประกาศณียบัตรผู้ช่วยพยาบาล)', 
                'อนุปริญญา', 'อื่นๆ (ป.ตรี)', 'อื่นๆ (ผู้ช่วยพยาบาล)'}
set_masterdocter = {'ปริญญาโท', 'ปริญญาเอก', "Master's degree", 'Other (OBGYN specalist lavel 1)', 
                    'Other (Residency)', 'Ph.D.', 'อื่นๆ (Internal Medicine)', 
                    'อื่นๆ (เฉพาะทาง)', 'อื่นๆ (วุฒิบัตร)', 'อื่นๆ (วว.ออร์โธปิดิกส์)', 
                    'อื่นๆ (วุฒิบัตรแสดงความรู้ความชำนาญในการประกอบวิชาชีพเภสัชกรรม สาขาเภสัชบำบัด)', 
                    'อื่นๆ (วุฒิบัตรผู้เชี่ยวชาญสาขาทันตกรรมทั่วไป)', 'อื่นๆ (วุฒิบัตรศัลยศาสตร์และแม็กซิลโลเฟเชียล)'}

list_degree = ((set_nan, 0), (set_primaryschool, 16), (set_middleschool, 19), 
               (set_highschool, 22), (set_bachelor,26), (set_masterdocter,40))

Create Series of Age-Education

In [10]:
ages = file.loc[:, 'อายุ']
educations = file.loc[:, 'วุฒิการศึกษา']
age_education_scores = []

for i,x in enumerate(educations):
    for y in list_degree:
        if x in y[0]:
            age_education_scores.append(getAgeEducationScore(ages[i], y[1]))
            
age_education_scores = pd.Series(age_education_scores, name='Age Education Score')

Age-Education Score Statistic

In [11]:
zero_score_count = age_education_scores.where(age_education_scores == 0).count()
one_score_count = age_education_scores.where(age_education_scores == 1).count()
three_score_count = age_education_scores.where(age_education_scores == 3).count()

print("Number of students who do not specify education:", zero_score_count)
print("Number of students who do not specify education:", one_score_count)
print("Number of students who do not specify education:", three_score_count)

Number of students who do not specify education: 123
Number of students who do not specify education: 4229
Number of students who do not specify education: 1775


Create Series of status

In [12]:
status = file.loc[:, 'สถานะ'].fillna("")
status = pd.Series(status ,name='Status')
status

0       ชำระเงิน
1       ชำระเงิน
2       ชำระเงิน
3       ชำระเงิน
4       ชำระเงิน
          ...   
6122    ชำระเงิน
6123    ชำระเงิน
6124    ชำระเงิน
6125    ชำระเงิน
6126    ชำระเงิน
Name: Status, Length: 6127, dtype: object

Provide a score to each user based on their purchase status

In [13]:
status_score = []
for x in status:
    if x == 'ชำระเงิน':
        status_score.append(8)
    if x == 'ไม่ผ่านการอนุมัติ':
        status_score.append(7)
    if x == 'ค้างชำระ':
        status_score.append(5)
status_score = pd.Series(status_score)

Purchase Status Score Statistics

In [14]:
five_score_count = status_score.where(status_score == 5).count()
seven_score_count = status_score.where(status_score == 7).count()
eight_score_count = status_score.where(status_score == 8).count()

print("Number of students who are in arrears:", five_score_count)
print("Number of students whose payment was not approved:", seven_score_count)
print("Number of students with payment approval:", eight_score_count)

Number of students who are in arrears: 531
Number of students whose payment was not approved: 124
Number of students with payment approval: 5472


Create Series of address

In [15]:
address = file.loc[:, 'ที่อยู่'].fillna("")
address = pd.Series(address ,name='Status')
address

0                                                        
1       125/27 ซ.3 ถ.ราษฎรยินดี ต.หน้าเมือง อ.เมือง จ....
2                                                        
3                                                        
4                                                        
                              ...                        
6122                                                     
6123                                                     
6124                                                     
6125                                                     
6126                                                     
Name: Status, Length: 6127, dtype: object

Provide a score to each user based on whether they provide address information or not

In [16]:
address_score = [ 1 if x == '' else 2 for x in address]
address_score = pd.Series(address_score)

Address Score Statistic

In [17]:
three_score_count = address_score.where(address_score == 1).count()
four_score_count = address_score.where(address_score == 2).count()

print("Number of students who did not fill address:", three_score_count)
print("Number of students who filled address:", four_score_count)

Number of students who did not fill address: 4005
Number of students who filled address: 2122


Convert list to pandas series

In [18]:
email_score = pd.Series(email_score)
age_education_scores = pd.Series(age_education_scores)
status_score = pd.Series(status_score)
address_score = pd.Series(address_score)

Create DataFrame by merging these 4 Series and calculate impressive level

In [6]:
user = file.loc[:, 'ชื่อ-นามสกุล (อังกฤษ)']
course = file.loc[:, 'หลักสูตรอบรมระยะสั้น']
d = {
    'User': user,
    'Course': course,
    'Email Score': email_score,
    'Age Education Score': age_education_scores,
    'Payment Score': status_score,
    'Address Score': address_score,
    'Point': email_score + status_score + address_score + age_education_scores,
    'Score': ( email_score + status_score + address_score + age_education_scores ) / 17
}
df = pd.DataFrame(d)
df.head()

NameError: name 'file' is not defined

Create user-course table

In [20]:
# all user, course, score have the same length
data = {
    'User': df['User'],
    'Course': df['Course'],
    'Score': df['Score'],
}

predata = pd.DataFrame(data)
predata

,User,Course,Score
0,PORPHAING JANTIP,หลักการและพื้นฐานของเครื่องมือทางรังสีวิทยา (B...,0.647059
1,THANAKORN DARASRISAK,การวินิจฉัยภาวะฉุกเฉินจากอุบัติเหตุ (Diagnosti...,0.823529
2,KARNJANA EAMTANG,การวินิจฉัยภาวะฉุกเฉินจากอุบัติเหตุ (Diagnosti...,0.764706
3,KARNJANA EAMTANG,การวินิจฉัยภาวะฉุกเฉินที่ไม่ได้เกิดจากอุบัติเห...,0.764706
4,THITIMUN VORATHONGCHAI,หลักการและพื้นฐานของเครื่องมือทางรังสีวิทยา (B...,0.764706
...,...,...,...
6122,KITTINON SANTASUP,เตรียมความพร้อมทางรังสีวิทยาสำหรับบุคลากรทางกา...,0.705882
6123,WITTAWAT SERMSRIPONG,เตรียมความพร้อมทางรังสีวิทยาสำหรับบุคลากรทางกา...,0.823529
6124,NON NAKKARA,เตรียมความพร้อมทางรังสีวิทยาสำหรับบุคลากรทางกา...,0.705882
6125,PHATTHARAPHON LIN,เตรียมความพร้อมทางรังสีวิทยาสำหรับบุคลากรทางกา...,0.705882


Calculate sparsity and csr matrix

In [21]:
# Pivot table by rotating course
data = predata.pivot_table(index='Course', columns='User', values='Score').fillna(0)
data.head()

User,AAIYANA MUSTAFA,ABDULFATTAH SENG,ABIGAIL LINE,ABRAM THANAWONG,ACHIRA TEAWSIRISUP,ACHIRAWIT AKSORNDIT,ACHIRAWIT PHUJOMCHAE,ACHIRAYA INNA,ACHIRAYA NUISEE,ACHIRAYA SUKPAISARN,...,YYY TTTT,ZABIDIN SENSANA,ZAR CHI LIN -,ZAYNUB NAEEM,ZE NYOI,ZHANGJINSHUO DONG,ZHENGHAO LIU,ZHENYI JIN,ZIWEN YONG,ZOE BESTMANNSMITH
Course,,,,,,,,,,,,,,,,,,,,,
Cancer Epidemiology and Prevention,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
Coaching Skill : Crafting a New You สำหรับนักศึกษา มหาวิทยาลัยเชียงใหม่,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
Coaching Skill : Crafting a New You สำหรับบุคลากร มหาวิทยาลัยเชียงใหม่,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
Development Studies & Social Research Specialization 2023,0.0,0.0,0.0,0.0,0.0,0.0,0.588235,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.764706,0.0,0.0,0.0,0.0,0.0
Digital Transformation: e-Document,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [22]:
# Item-User Rated Matrix
data_mtx = csr_matrix(data.values)

Euclidean Distance & Cosine Similarity

In [23]:
# model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=int(np.around(math.sqrt(len(courses))))).fit(data_mtx)
model_knn = NearestNeighbors(metric='cosine').fit(data_mtx)
model_knn

NearestNeighbors(metric='cosine')

In [24]:
def recommender_knn(course_name, n_recommendations):
    idx = process.extractOne(course_name, courses)[2]
    print('Selected movie:', courses[idx], 'Index:', idx)
    distances, indices = model_knn.kneighbors(data_mtx[idx], n_neighbors=n_recommendations, return_distance=True)
    recommendations = [courses[i].where(i!=idx) for i in indices]
    recommended_courses = recommendations[0][1:]
    course_distances = distances[0][1:]
    d = {
        'Course': recommended_courses,
        'Cosine Distance': course_distances
    }
    results = pd.DataFrame(data=d)
    return results

In [25]:
recommender_knn('การวินิจฉัยภาวะฉุกเฉินจากอุบัติเหตุ (Diagnostic Radiology of Traumatic Emergency)', 100)

Selected movie: การวินิจฉัยภาวะฉุกเฉินจากอุบัติเหตุ (Diagnostic Radiology of Traumatic Emergency) Index: 42


,Course,Cosine Distance
43,การวินิจฉัยภาวะฉุกเฉินที่ไม่ได้เกิดจากอุบัติเห...,0.245415
131,หลักการและพื้นฐานของเครื่องมือทางรังสีวิทยา (B...,0.720149
110,รังสีวิทยาวินิจฉัย,0.771847
152,เตรียมความพร้อมทางรังสีวิทยาสำหรับบุคลากรทางกา...,0.819078
124,สารสนเทศทางสาธารณสุข 2566 (Public Health Infor...,0.931377
...,...,...
140,หลักเศรษฐศาสตร์สาธารณสุข (Principle of Health ...,1.000000
141,หุ่นยนต์วิจักษณ์ (Robotics Appreciation) 2566 ...,1.000000
142,หุ่นยนต์วิจักษณ์ (Robotics Appreciation) 2566 ...,1.000000
118,วิทยาศาสตร์ขั้นสูงในการดูแลผิวพรรณและสุขภาวะผิ...,1.000000


Predata for hybrid recommendation

In [26]:
def recommender_knn_all_courses(course_name):
    model_knn.fit(data_mtx)
    idx = process.extractOne(course_name, courses)[2]
    print('Selected movie:', courses[idx], 'Index:', idx)
    distances, indices = model_knn.kneighbors(data_mtx[idx], n_neighbors=len(courses))
    recommendations = [courses[i].where(i!=idx) for i in indices]
    recommended_courses = recommendations[0][1:]
    scores = 1 - distances
    course_distances = scores[0][1:]
    d = {
        'Course': recommended_courses,
        'Score': course_distances
    }
    results = pd.DataFrame(data=d)
    results = results.sort_index().rename_axis('Index')
    return results

In [27]:
recommender_knn_all_courses('การวินิจฉัยภาวะฉุกเฉินจากอุบัติเหตุ (Diagnostic Radiology of Traumatic Emergency)')

Selected movie: การวินิจฉัยภาวะฉุกเฉินจากอุบัติเหตุ (Diagnostic Radiology of Traumatic Emergency) Index: 42


,Course,Score
Index,,
0,Cancer Epidemiology and Prevention,0.000000
1,Coaching Skill : Crafting a New You สำหรับนักศ...,0.000000
2,Coaching Skill : Crafting a New You สำหรับบุคล...,0.000000
3,Development Studies & Social Research Speciali...,0.017835
4,Digital Transformation: e-Document,0.000000
...,...,...
163,เวชศาสตร์ครอบครัวขั้นสูง,0.000000
164,เวชศาสตร์ฟื้นฟูสำหรับแพทย์ฝึกอบรมเวชศาสตร์ฟื้น...,0.000000
165,แปลงจุดแข็ง เป็นคุณค่า สร้างชีวิตสู่ความสุข (...,0.000000


In [28]:
def recommender_knn_by_user(user_name, n_recommendations):
    df = {
        'User': pd.Series(file['ชื่อ-นามสกุล (อังกฤษ)']),
        'Course': pd.Series(file['หลักสูตรอบรมระยะสั้น'])
    }
    
    user_course = pd.DataFrame(df)
    selected_user_name = user_course.loc[user_course['User'] == user_name]
    selected_courses = selected_user_name['Course']
    
    recommended_courses = [ recommender_knn_all_courses(x) for x in selected_courses]
    
    # pre dataframe
    df = pd.DataFrame({
        'Course': [],
        'Score': []
    }).rename_axis('Index')
    
    for x in recommended_courses:
        df = df._append(x)
    df =  df.sort_values('Score', ascending=False).drop_duplicates('Course')
    return df.head(n_recommendations)

Items show permanence whereas, people change with time
Items are fewer in numbers to deal with. Which leads to smaller similarity matrix. Amazon and Netflix use it!
Better for New users:
— Him selecting just one item will let us provide recommendations
— But for user based, new user has to wait until next build of similarity matrix (which is the only computational part of the framework)

In [29]:
recommender_knn_by_user('PORPHAING JANTIP', 10)

Selected movie: หลักการและพื้นฐานของเครื่องมือทางรังสีวิทยา (Basic Principle of Diagnostic Radiology Imaging Instruments) Index: 131
Selected movie: การวินิจฉัยภาวะฉุกเฉินที่ไม่ได้เกิดจากอุบัติเหตุ (Diagnostic Radiology of Non-Traumatic Emergency) Index: 43
Selected movie: การวินิจฉัยภาวะฉุกเฉินจากอุบัติเหตุ (Diagnostic Radiology of Traumatic Emergency) Index: 42


,Course,Score
Index,,
43,การวินิจฉัยภาวะฉุกเฉินที่ไม่ได้เกิดจากอุบัติเห...,0.754585
42,การวินิจฉัยภาวะฉุกเฉินจากอุบัติเหตุ (Diagnosti...,0.754585
152,เตรียมความพร้อมทางรังสีวิทยาสำหรับบุคลากรทางกา...,0.326572
131,หลักการและพื้นฐานของเครื่องมือทางรังสีวิทยา (B...,0.279851
110,รังสีวิทยาวินิจฉัย,0.258932
124,สารสนเทศทางสาธารณสุข 2566 (Public Health Infor...,0.069610
163,เวชศาสตร์ครอบครัวขั้นสูง,0.061968
41,การวิจัยแบบผสมผสานทางสุขภาพ (Mixed Methods Res...,0.032926
85,ความรู้พื้นฐานในการทำวิจัยเพื่อพัฒนางาน (Resea...,0.027183
